# Big Data Boss

### This notebook contains
1) Work on the Data Collected byu webscraping(Very Very less data)

2) Work on the Data taken from Internet

#### Data collected by me (Very very small data)

In [ ]:
import requests
import bs4
import csv

In [ ]:
res = requests.get("https://www.imdb.com/list/ls025933303/")

In [ ]:
soup = bs4.BeautifulSoup(res.text,"lxml")

In [ ]:
head1= soup.select(".lister-item-header")

In [ ]:
head1

In [ ]:
Name,Year =[],[]
for i in soup.select('h3 > a'):
    Name.append(i.text)
for i in range(len(head1)):
    Year.append(head1[i].select("span")[1].text)

In [ ]:
Name

In [ ]:
Year

In [ ]:
head2= soup.select(".runtime")

In [ ]:
runtime=[]
for i in head2:
    runtime.append(i.text)

In [ ]:
runtime

In [ ]:
rating=[]
i=0
while i < 575:
    print(soup.select(".ipl-rating-star__rating")[i].text)
    rating.append(soup.select(".ipl-rating-star__rating")[i].text)
    i += 23

In [ ]:
rating

In [ ]:
term = soup.select(".genre")

In [ ]:
genre=[]
for i in range(len(term)):
    genre.append(term[i].text) 

In [ ]:
genre

In [ ]:
data = []

In [ ]:
for i in range(len(Name)):
    data.append({'Name' : Name[i],'Year':Year[i],'Runtime': runtime[i],'Rating': rating[i],'Genre': genre[i]})
        

In [ ]:
csv_columns = ['Name','Year','Runtime','Rating','Genre']

csv_file = "data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in data:
            writer.writerow(data)
except IOError:
    print("I/O error")  

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
df["Genre"] = df["Genre"].apply(lambda x: x.split()[0])
df["Genre"] = df["Genre"].apply(lambda x: x.replace(r"/", ""))
df["Genre"] = df["Genre"].apply(lambda x: x.replace(r",", ""))

df["Genre"] = df["Genre"].apply(lambda x: x.replace(r"\n", ""))
df.sample(5)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.title("Genres")
sns.countplot(y = df["Genre"])
plt.xlabel("Count")
plt.ylabel("Genres")
plt.show()

In [ ]:
plt.title("Rating distribution")
sns.distplot(df["Rating"])
plt.xlabel("Rating")
plt.show()

In [ ]:
df['Runtime'] = df["Runtime"].apply(lambda x: x.split()[0])

In [ ]:
df['Runtime'].astype(int)

In [ ]:
plt.title("Runtime distribution")
sns.distplot(df["Runtime"])
plt.xlabel("Runtime")
plt.show()

In [ ]:
df["Year"] = df["Year"].apply(lambda x: x[1:5])


In [ ]:
df.Year.astype(int)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.title("Year")
sns.countplot(y = df["Year"])
plt.xlabel("Count")
plt.ylabel("Year")
plt.show()

In [ ]:
df.dtypes

In [ ]:
for i in df.columns:
    print(f'length of unique values in {i}',len(set(df[i])))
    print(f'some of the unique values in {i}',list(set(df[i]))[0:5])
    print('---------------------------------------------------------')

In [ ]:
df.loc[df['Genre'] =='Drama', 'Genre'] = 0
df.loc[df['Genre'] =='Action', 'Genre'] = 1
df.loc[df['Genre'] =='Comedy', 'Genre'] = 2

In [ ]:
for i in ['Year','Runtime','Genre']:
    df[i] = pd.to_numeric(df[i],errors='coerce')

In [ ]:
f, ax = plt.subplots(figsize=(9,9))
sns.heatmap(df.corr(), annot=True ,linewidth=0.5, fmt='.1f',ax=ax)

## Data Visualisations of data collected from internet 

In [ ]:
data = pd.read_csv('../input/kollywood-movie-dataset-2011-2017/Kollywood Movie Dataset (2011 - 2017).csv')

In [ ]:
data

In [ ]:
data.isna().sum()

Here the 7 rating values are null lets fill them with the average value of Rating

In [ ]:
avg_rating = np.sum(data.Rating)/(416-7)

In [ ]:
data['Rating'].fillna(value=avg_rating, inplace=True)

In [ ]:
data.head()

In [ ]:
data = data.drop(['Language'],axis =1)

In [ ]:
for i in data.columns:
    print(f'length of unique values in {i}',len(set(data[i])))
    print(f'some of the unique values in {i}',list(set(data[i]))[0:5])
    print('-'*100)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.title("Year")
sns.countplot(y = data['Release Year'])
plt.xlabel("Count")
plt.ylabel("Year")
plt.show()

In [ ]:
data

In [ ]:
plt.title("Rating")
sns.distplot(data["Rating"])
plt.xlabel("Rating")
plt.show()

In [ ]:
data['Genre'] = data['Genre'].apply(lambda x: x.split(' ')[0])

In [ ]:
data["Genre"] = data["Genre"].apply(lambda x: x.replace(r"/", ""))
data["Genre"] = data["Genre"].apply(lambda x: x.replace(r",", ""))

data["Genre"] = data["Genre"].apply(lambda x: x.replace(r"\n", ""))

In [ ]:
data.Genre.unique()

Using NLP

In [ ]:
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


In [ ]:
def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    texts_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            texts_clean.append(stem_word)


    return texts_clean

In [ ]:
def build_freqs(texts, ys):

    yslist = np.squeeze(ys).tolist()

    freqs = {}
    for y, text in zip(yslist, texts):
        for word in process_text(text):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [ ]:
data['Status'] = np.select([(data['Rating'] >= 5.5),(data['Rating'] < 5.5)],[1,0])

In [ ]:
data.columns

In [ ]:
data['text'] = data.apply(lambda row: row.Director+ 
                                  (row.Plot), axis = 1) 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = data.Status
data =data.drop(['Status'],axis=1)

In [ ]:
X_train = data[0:332]
X_test = data[332:]
y_train= y[0:332]
y_test = y[332:]

In [ ]:
X_train.columns

In [ ]:
freqs_buil = build_freqs(X_train.text,y_train)

In [ ]:
def extract_features(text, freqs):
    word_l = process_text(text)
    
    x = np.zeros((1, 3)) 
    
    x[0,0] = 1 
    
    
    for word in word_l:
        
        x[0,1] += freqs.get((word,1.0), 0)
        
        x[0,2] += freqs.get((word,0.0), 0)
        
    assert(x.shape == (1, 3))
    return x


In [ ]:
X = np.zeros((len(X_train), 3))
for i in range(0,len(X_train)):
    X[i,:]= extract_features(X_train.text[i],freqs_buil)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, plot_confusion_matrix, accuracy_score


In [ ]:
model = LogisticRegression(penalty= 'l2' ,random_state= 42 ,max_iter=20,solver='liblinear',class_weight= 'balanced')
model.fit(X,y_train)

In [ ]:
X_t = np.zeros((len(X_test), 3))


In [ ]:
i=332
j=0
while i <416:
    X_t[j,:]= extract_features(X_test.text[i],freqs_buil)
    i+=1
    j+=1

In [ ]:
y_pred = model.predict(X_t)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
plot_confusion_matrix(model, X_t, y_test,labels=[0,1],normalize= 'true')
